In [5]:
import json
from collections import Counter
import regex as re
import os
# from transformers import AutoTokenizer #4.25.1 torch 1.7.1
import pandas as pd
import random
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer
import pyphen


In [ ]:
path='./SlovakBabyLM/Curricullum_learning/'
file_to_train='strict_model_ordered_data_reverse'
file_to_save='1_full_data__without_ord__without_mask'

# paths=[f'{path}strict_model_individual_datasets/wiki_full/wiki_full.json']
# txt_paths=[f'{path}strict_model_results/7_full_data__sum_freq_gramma__max_freq/final_not_masked_text.txt']
# txt_paths=[f'{path}strict_model_ordered_data_results/1_full_data__without_ord__without_mask/randomize.txt']

# strict_model
# paths=[f'{path}strict_model/childes/childes_reordered_sum_freq_gramma.json',
#  f'{path}strict_model/subs/subs_reordered_sum_freq_gramma.json',
#  f'{path}strict_model/fairytales/fairytales_reordered_sum_freq_gramma.json',
#  f'{path}strict_model/lit/lit_reordered_sum_freq_gramma.json',
#  f'{path}strict_model/edu/edu_reordered_sum_freq_gramma.json',
#  f'{path}strict_model/wiki/wiki_reordered_sum_freq_gramma.json']

#strict_model_ordered
# paths=[f'{path}strict_model_ordered_data/subs/subs.json',
#  f'{path}strict_model_ordered_data/childes/childes.json',
#  f'{path}strict_model_ordered_data/wiki_full/wiki_full.json',
#  f'{path}strict_model_ordered_data/lit/lit.json',
#  f'{path}strict_model_ordered_data/fairytales_full/fairytales_full.json',
#  f'{path}strict_model_ordered_data/edu/edu.json']

# paths=[f'{path}strict_model_ordered_data_full_data/strict_model_ordered_data_full_data_processed_reordered_sum_freq_gramma.json']

# strict_model_ordered_reverse
paths=[f'{path}strict_model_ordered_data_reverse_full_data/strict_model_ordered_data_reverse_full_data_processed.json']


In [ ]:
test_tokenizer= Tokenizer.from_file(f"{path}tok_bpe/{file_to_train}_BPE/tokenizer.json")
hf_tokenizer = PreTrainedTokenizerFast(tokenizer_object=test_tokenizer, return_special_tokens_mask=True, mask_token='[MASK]')


### Create unmasked text and save it 

In [10]:
def chunk_text(text, max_tokens=128):
    if text:
        encoded = hf_tokenizer.encode_plus(text, add_special_tokens=False, return_offsets_mapping=True)
        offsets = encoded["offset_mapping"] 
        divisible_offsets = [offsets[i] for i in range(max_tokens, len(offsets), max_tokens)]
        if divisible_offsets:
            chunks = []
            start_offset=0
            for offset in divisible_offsets:
                chunks.append(text[start_offset:offset[-1]])
                start_offset=offset[-1]
            chunks.append(text[offset[-1]:])  
            return chunks
        return [text]

In [11]:
def create_text_from_json(paths,shuffle):
    text=''
    for file_path in paths:
        print(file_path)
        with open(file_path, "r", encoding="utf8") as f:
            data = json.load(f)
        if shuffle:
            data = random.sample(data, len(data))        
        for i in data:
            chunks=chunk_text(i['page'])
            source=' 𡨸 '.join(chunks)
            text=text+source 
    return text

In [12]:
def create_text_from_text(paths,shuffle):
    for file_path in paths:
        print(file_path)
        with open(file_path, 'r') as file:
            content = file.read()
        if shuffle:
            content = random.sample(content, len(content))
        chunks=chunk_text(content)
        text=' 𡨸 '.join(chunks)
    return text

Create unmasked text: for file and folders

In [ ]:
text=create_text_from_json(paths,shuffle=True)  

In [ ]:
text=create_text_from_text(txt_paths,shuffle=True)

NameError: name 'create_text_from_text' is not defined

Save text

In [ ]:
path_to_not_mask=f'{path}{file_to_train}_results/{file_to_save}/final_not_masked_text.txt'
with open(path_to_not_mask,'a',encoding="utf8") as f:
    f.write(f'{text}')

### Create masked text 

In [ ]:
text=''
for file_path in txt_paths:
    print(file_path)
    with open(file_path, "r", encoding="utf8") as f:
        data = f.read()
        text=text+data
        data=''


In [18]:
def select_mask(evaluation_list,proc_dspan):
    max_value = max(evaluation_list)
    indices = [i for i, val in enumerate(evaluation_list) if val == max_value]
    mask_id=random.choice(indices)
    return proc_dspan[mask_id]

In [39]:
list_freq=[]
list_word_freq=[]
masked_dspan_all=[]
number=0
full_word_freq = Counter()

pattern = r"\w+('\w+)?|[^\w\s]"
source = [(match.group(), match.start(), match.end()) for match in re.finditer(pattern, text)]

for w in source:
    if not w[0] or re.match(r'^\W+$', w[0]) or w[0] in ["URL","LINK","TEL"] or w[0] =='𡨸':
        continue
    number+=1
    word_freq = Counter([w[0]])
    full_word_freq.update(word_freq) 
    list_freq.append(full_word_freq[w[0]])
    dspan=(w[1],w[2])
    list_word_freq.append(dspan)
    if number==4:
        masked_dspan=select_mask(list_freq,list_word_freq)
        masked_dspan_all.append(masked_dspan)
        list_word_freq=[]
        list_freq=[]
        number=0
masked_dspan_all.insert(0,(0,0))

KeyboardInterrupt: 

In [ ]:
print(len(masked_dspan_all))

In [ ]:
mask_words=[]
masked_text=''
path_to_save_masked_text=f'{path}{file_to_train}_results/{file_to_save}/final_masked_text.txt'
path_to_save_masked_words=f'{path}{file_to_train}_results/{file_to_save}/masked_words.txt'

last_mask= masked_dspan_all[-1][-1]
len_text= len(text)


len_text,len_text
for i_dspan in range(1,len(masked_dspan_all)):
    prev_start, prev_end = masked_dspan_all[i_dspan-1]
    start, end = masked_dspan_all[i_dspan]
    count_mask=len(hf_tokenizer.encode_plus(text[start:end], add_special_tokens=False, return_offsets_mapping=True)['input_ids'])
    mask_words.append(text[start:end])
    part_for_masking=text[prev_end:end]
    index = part_for_masking.rfind(text[start:end])
    mask="[MASK]"*count_mask
    part_text=part_for_masking[:index] + mask + part_for_masking[index + len(text[start:end]):]
    masked_text+= part_text
masked_text=masked_text+text[last_mask:len_text]

with open(path_to_save_masked_text,'a',encoding="utf8") as f:
    f.write(masked_text)
with open(path_to_save_masked_words, 'w',encoding="utf8") as output:
    for row in mask_words:
        output.write(str(row) + '\n')